In [24]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [26]:
from tensorflow.keras.datasets import mnist

In [27]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:(RGB則3channel)

    (28,28) --> (28, 28, 1)

In [28]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [29]:

x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [30]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [31]:
model = Sequential()

In [32]:
model.add(Conv2D(8, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [33]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [34]:
model.add(Conv2D(16, (3,3), padding='same',
                activation='relu'))

In [35]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [36]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [37]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [38]:
model.add(Flatten())

In [39]:
model.add(Dense(20, activation='relu'))
＃變成神經元

In [40]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)              

#### 組裝

In [42]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [43]:
model.fit(x_train, y_train, batch_size=128, epochs=12)
#epoch為訓練幾次，accuracy盡量大於95%

Epoch 1/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0899 - accuracy: 0.1113
Epoch 2/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0897 - accuracy: 0.1405
Epoch 3/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0894 - accuracy: 0.1760
Epoch 4/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0889 - accuracy: 0.2208
Epoch 5/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0878 - accuracy: 0.2876
Epoch 6/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0844 - accuracy: 0.3684
Epoch 7/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0678 - accuracy: 0.5260
Epoch 8/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0324 - accuracy: 0.8017
Epoch 9/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0181 - accuracy: 0.8863
Epoch 10/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0142 - accuracy: 0.9083

### Step 3. 預測

In [44]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [45]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0101 - accuracy: 0.9347


In [46]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 93.47%


In [47]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [48]:
from ipywidgets import interact_manual

In [49]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [50]:
model.save('myCNNmodel.h5')